In [ ]:
!git clone https://github.com/Madhudorai/dcase2024.git

In [ ]:
!pip install pyroomacoustics

In [3]:
import numpy as np
import soundfile as sf
from scipy import signal
import os
import scipy.special as sp
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.special import factorial, lpmv
import pyroomacoustics as pra

In [ ]:
import os
print(os.getcwd())
os.chdir('/content/dcase2024')
print(os.getcwd())

In [5]:
import MicrophoneArray
import plot_tools
from tools import sph2cart, soundread, setRoom, ssl_SHmethod_broad
from plot_tools import plot_wave, plot_stft, plot_option

In [ ]:
MicArray32 = MicrophoneArray.EigenmikeEM32()
name = MicArray32.getname()
array_type = MicArray32.gettype()
thetas = MicArray32._thetas
phis = MicArray32._phis
radius = MicArray32._radius
weights = MicArray32._weights
num_elements = MicArray32._numelements
directivity = MicArray32._directivity

# Initialize the list of mic_position in spherical harmonic domain
mic_pos_sph = []

# for each theta and phi, create a new coordinate which contains all parameters
# to the list of mic_pos
for theta, phi in zip(thetas, phis):
    mic_pos_sph.append([radius, theta, phi])
mic_pos_sph = np.array(mic_pos_sph)
print("Name:", name)
print("Array Type:", array_type)
print("Thetas:", thetas)
print("Phis:", phis)
print("Radius:", radius)
print("Weights:", weights)
print("Number of Elements:", num_elements)
print("Directivity:", directivity)

In [7]:
### The room properties
room_dim = np.array([9, 6, 3])
ori_coor = room_dim/2

In [ ]:
### Sound Source
mic_signals_speech, fs, num_channels = soundread('/content/dcase2024/Data/speech@24kHz.wav')
plot_wave(mic_signals_speech, fs)
plot_stft(mic_signals_speech, fs)
Theta_l = 50/180 * np.pi
Phi_l = 230/180 * np.pi
source_pos_car = 50*radius * np.array([np.sin(Theta_l) * np.cos(Phi_l), np.sin(Theta_l) * np.sin(Phi_l), np.cos(Theta_l)])
source_pos_car_cal = source_pos_car + ori_coor
source_pos_car_list = [source_pos_car_cal]
signal_list = [mic_signals_speech]


In [9]:
mic_pos_car = [sph2cart(r,theta,phi) for r,theta,phi in mic_pos_sph]
mic_pos_car = np.array(mic_pos_car)
mic_pos_car_cal = mic_pos_car + ori_coor
mic_arrays_car = [mic_pos_car_cal]

In [ ]:
anechoic_room = setRoom(room_dim, mic_arrays_car, source_pos_car_list, signal_list, "Anechoic")
reverberant_room ,rt60_est= setRoom(room_dim, mic_arrays_car, source_pos_car_list, signal_list, "Reverb",0.4)
print(f'The simulated rt60 is', rt60_est)

In [ ]:
plot_option(reverberant_room, mic_arrays_car, source_pos_car_list)

In [ ]:
file_path = "/content/dcase2024/Reverberant/Array_output_0/source0.wav"
signal_processed = pra.create_noisy_signal(file_path,15)[0]
plot_wave(signal_processed[:, 0], fs)
plot_stft(signal_processed[:, 0], fs)

In [ ]:
out_DAS, pos_est_DAS= ssl_SHmethod_broad(signal_processed,fs,mic_pos_sph,Theta_l,Phi_l,"DAS","rigid","2D",resolution = 1)